# ADS Chappa

## Preprocessing Part

Importing packages

In [1]:
import numpy as np
import pandas as pd

Reading file

In [2]:
data = pd.read_csv('Customer_Sales_Transactional_Data_CSV.txt')

In [3]:
df = data.copy()

In [4]:
df.head()

,SALES_dATE,CUSTOMER_ID,SALES_AMOUNT
0,10/18/2014,34810920,205.44
1,9/22/2014,1026037818,51.36
2,9/30/2014,1095693062,222.56
3,10/25/2014,15142688,171.20
4,10/18/2014,1022245368,171.20


Converting Sales_Date to datetime format


In [5]:
df['SALES_DATE'] = ''

In [6]:
df.loc[:, ('SALES_DATE')] = pd.to_datetime(df['SALES_dATE'])

In [7]:
df.drop('SALES_dATE', axis=1, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058198 entries, 0 to 1058197
Data columns (total 3 columns):
CUSTOMER_ID     1058198 non-null int64
SALES_AMOUNT    1058198 non-null float64
SALES_DATE      1058198 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 24.2 MB


In [9]:
df.head()

,CUSTOMER_ID,SALES_AMOUNT,SALES_DATE
0,34810920,205.44,2014-10-18
1,1026037818,51.36,2014-09-22
2,1095693062,222.56,2014-09-30
3,15142688,171.20,2014-10-25
4,1022245368,171.20,2014-10-18


Calculating weeks

In [10]:
df['SALES_WEEK'] = pd.DatetimeIndex(df['SALES_DATE']).week

Converting Weeks to 5, 4, 3, 2, 1, and 0 format

In [11]:
df['SALES_WEEK'] = abs((df['SALES_WEEK']%9) - 7)

/home/usama/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


We need to create following features for each customer on each week:

* Historic_Visits
* Historic_Sales
* STD_SALESAMOUNT
* VARIATION_SALESAMOUNT
* MAX_SALESAMOUNT
* MIN_SALESAMOUNT

Note: In this project we will be creating Churn feature from Week-0

In [12]:
weeks = df.groupby(['CUSTOMER_ID', 'SALES_WEEK']).agg({'SALES_AMOUNT':{'HISTORIC_VISITS': 'count', 'HISTORIC_SALES':'sum', 'STD_SALESAMOUNT':'std', 'VARIATION_SALESAMOUNT':'var', 'MIN_SALESAMOUNT': 'min', 'MAX_SALESAMOUNT': 'max'}}).reset_index()

/home/usama/.local/lib/python3.5/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [13]:
weeks.columns = weeks.columns.droplevel(0)

In [14]:
weeks.columns

Index(['', '', 'MAX_SALESAMOUNT', 'VARIATION_SALESAMOUNT', 'STD_SALESAMOUNT',
       'HISTORIC_VISITS', 'MIN_SALESAMOUNT', 'HISTORIC_SALES'],
      dtype='object')

In [15]:
weeks.columns = ['CUSTOMER_ID', 'WEEK', 'HISTORIC_VISITS', 'HISTORIC_SALES', 'STD_SALESAMOUNT',
       'VARIATION_SALESAMOUNT', 'MIN_SALESAMOUNT', 'MAX_SALESAMOUNT']

In [16]:
weeks.head()

,CUSTOMER_ID,WEEK,HISTORIC_VISITS,HISTORIC_SALES,STD_SALESAMOUNT,VARIATION_SALESAMOUNT,MIN_SALESAMOUNT,MAX_SALESAMOUNT
0,488,3,171.20,NaN,NaN,1,171.20,171.20
1,5194,2,599.20,NaN,NaN,1,599.20,599.20
2,5398,0,51.36,NaN,NaN,1,51.36,51.36
3,5398,2,51.36,NaN,NaN,1,51.36,51.36
4,5398,4,51.36,NaN,NaN,1,51.36,51.36


In [17]:
weeks = weeks.pivot(index='CUSTOMER_ID', columns='WEEK')

In [18]:
weeks.head()

HISTORIC_VISITS                                       \
WEEK                      0      1       2      3       4      5   
CUSTOMER_ID                                                        
488                     NaN    NaN     NaN  171.2     NaN    NaN   
5194                    NaN    NaN  599.20    NaN     NaN    NaN   
5398                  51.36    NaN   51.36    NaN   51.36    NaN   
6930                  85.60   85.6  171.20   85.6   85.60    NaN   
7260                 171.20  171.2  171.20  256.8  171.20  171.2   

            HISTORIC_SALES                                   ...    \
WEEK                     0          1            2        3  ...     
CUSTOMER_ID                                                  ...     
488                    NaN        NaN          NaN      NaN  ...     
5194                   NaN        NaN          NaN      NaN  ...     
5398                   NaN        NaN          NaN      NaN  ...     
6930                  0.00        NaN  3663.680000      NaN  ...     
7260               3663.68  2637.8496   390.792533  7327.36  ...     

            MIN_SALESAMOUNT                      MAX_SALESAMOUNT          \
WEEK                      2      3      4      5               0       1   
CUSTOMER_ID                                                                
488                     NaN  171.2    NaN    NaN             NaN     NaN   
5194                 599.20    NaN    NaN    NaN             NaN     NaN   
5398                  51.36    NaN  51.36    NaN           51.36     NaN   
6930                  85.60   85.6  85.60    NaN          256.80   85.60   
7260                 136.96   85.6  85.60  171.2          256.80  582.08   

                                           
WEEK              2      3       4      5  
CUSTOMER_ID                                
488             NaN  171.2     NaN    NaN  
5194         599.20    NaN     NaN    NaN  
5398          51.36    NaN   51.36    NaN  
6930         256.80   85.6  171.20    NaN  
7260         445.12  513.6  256.80  171.2  

[5 rows x 36 columns]

In [19]:
weeks.columns

MultiIndex(levels=[['HISTORIC_VISITS', 'HISTORIC_SALES', 'STD_SALESAMOUNT', 'VARIATION_SALESAMOUNT', 'MIN_SALESAMOUNT', 'MAX_SALESAMOUNT'], [0, 1, 2, 3, 4, 5]],
           labels=[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5], [0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]],
           names=[None, 'WEEK'])

In [20]:
weeks.columns.droplevel()

Int64Index([0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3,
            4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5],
           dtype='int64', name='WEEK')

In [21]:
weeks.columns = weeks.columns.droplevel()

In [22]:
weeks.columns

Int64Index([0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3,
            4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5],
           dtype='int64', name='WEEK')

In [23]:
weeks.columns = ['W0_HISTORIC_VISITS', 'W1_HISTORIC_VISITS', 'W2_HISTORIC_VISITS', 'W3_HISTORIC_VISITS', 'W4_HISTORIC_VISITS', 'W5_HISTORIC_VISITS', 'W0_HISTORIC_SALES', 'W1_HISTORIC_SALES', 'W2_HISTORIC_SALES', 'W3_HISTORIC_SALES', 'W4_HISTORIC_SALES', 'W5_HISTORIC_SALES', 'W0_STD_SALESAMOUNT', 'W1_STD_SALESAMOUNT', 'W2_STD_SALESAMOUNT', 'W3_STD_SALESAMOUNT', 'W4_STD_SALESAMOUNT', 'W5_STD_SALESAMOUNT', 'W0_VARIATION_SALES_AMOUNT', 'W1_VARIATION_SALES_AMOUNT', 'W2_VARIATION_SALES_AMOUNT', 'W3_VARIATION_SALES_AMOUNT', 'W4_VARIATION_SALES_AMOUNT', 'W5_VARIATION_SALES_AMOUNT', 'W0_MIN_SALESAMOUNT', 'W1_MIN_SALESAMOUNT', 'W2_MIN_SALESAMOUNT', 'W3_MIN_SALESAMOUNT', 'W4_MIN_SALESAMOUNT', 'W5_MIN_SALESAMOUNT', 'W0_MAX_SALESAMOUNT', 'W1_MAX_SALESAMOUNT', 'W2_MAX_SALESAMOUNT', 'W3_MAX_SALESAMOUNT', 'W4_MAX_SALESAMOUNT', 'W5_MAX_SALESAMOUNT']
weeks.head()

,W0_HISTORIC_VISITS,W1_HISTORIC_VISITS,W2_HISTORIC_VISITS,W3_HISTORIC_VISITS,W4_HISTORIC_VISITS,W5_HISTORIC_VISITS,W0_HISTORIC_SALES,W1_HISTORIC_SALES,W2_HISTORIC_SALES,W3_HISTORIC_SALES,...,W2_MIN_SALESAMOUNT,W3_MIN_SALESAMOUNT,W4_MIN_SALESAMOUNT,W5_MIN_SALESAMOUNT,W0_MAX_SALESAMOUNT,W1_MAX_SALESAMOUNT,W2_MAX_SALESAMOUNT,W3_MAX_SALESAMOUNT,W4_MAX_SALESAMOUNT,W5_MAX_SALESAMOUNT
CUSTOMER_ID,,,,,,,,,,,,,,,,,,,,,
488,NaN,NaN,NaN,171.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,171.2,NaN,NaN,NaN,NaN,NaN,171.2,NaN,NaN
5194,NaN,NaN,599.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,599.20,NaN,NaN,NaN,NaN,NaN,599.20,NaN,NaN,NaN
5398,51.36,NaN,51.36,NaN,51.36,NaN,NaN,NaN,NaN,NaN,...,51.36,NaN,51.36,NaN,51.36,NaN,51.36,NaN,51.36,NaN
6930,85.60,85.6,171.20,85.6,85.60,NaN,0.00,NaN,3663.680000,NaN,...,85.60,85.6,85.60,NaN,256.80,85.60,256.80,85.6,171.20,NaN
7260,171.20,171.2,171.20,256.8,171.20,171.2,3663.68,2637.8496,390.792533,7327.36,...,136.96,85.6,85.60,171.2,256.80,582.08,445.12,513.6,256.80,171.2


In [24]:
weeks[['W1_HISTORIC_VISITS', 'W1_HISTORIC_SALES', 'W1_STD_SALESAMOUNT', 'W1_VARIATION_SALES_AMOUNT', 'W1_MIN_SALESAMOUNT', 'W1_MAX_SALESAMOUNT']].head()

,W1_HISTORIC_VISITS,W1_HISTORIC_SALES,W1_STD_SALESAMOUNT,W1_VARIATION_SALES_AMOUNT,W1_MIN_SALESAMOUNT,W1_MAX_SALESAMOUNT
CUSTOMER_ID,,,,,,
488,NaN,NaN,NaN,NaN,NaN,NaN
5194,NaN,NaN,NaN,NaN,NaN,NaN
5398,NaN,NaN,NaN,NaN,NaN,NaN
6930,85.6,NaN,NaN,1.0,85.60,85.60
7260,171.2,2637.8496,51.36,4.0,68.48,582.08


In addition to each week feature, we also need below overall features against each customer:

* Historic_Visits
* Historic_Sales
* STD_SALESAMOUNT
* VARIATION_SALESAMOUNT
* MAX_SALESAMOUNT
* MIN_SALESAMOUNT
* FIRST_VISIT
* LAST_VISIT

Note: These features will be calculated on Week-5 to Week-1, we will not include Week-0 as we are deriving CHURN feature from it

In [25]:
overall_detail = df[df['SALES_WEEK'] > 0].groupby('CUSTOMER_ID').agg({'SALES_AMOUNT': {'HISTORIC_VISITS': 'count', 'HISTORIC_SALES': 'sum', 'STD_SALESAMOUNT': 'std', 'VARIATION_SALESAMOUNT': 'var', 'MAX_SALESAMOUNT': 'max', 'MIN_SALESAMOUNT': 'min'}, 'SALES_DATE':{'LAST_VISIT': 'max', 'FIRST_VISIT': 'min'}})

In [26]:
overall_detail.head()

SALES_AMOUNT                                        \
            MAX_SALESAMOUNT VARIATION_SALESAMOUNT STD_SALESAMOUNT   
CUSTOMER_ID                                                         
488                  171.20                   NaN             NaN   
5194                 599.20                   NaN             NaN   
5398                  51.36              0.000000        0.000000   
6930                 171.20           1221.226667       34.946054   
7260                 256.80           2803.721705       52.950181   

                                                            SALES_DATE  \
            HISTORIC_VISITS MIN_SALESAMOUNT HISTORIC_SALES FIRST_VISIT   
CUSTOMER_ID                                                              
488                       1          171.20         171.20  2014-10-03   
5194                      1          599.20         599.20  2014-10-09   
5398                      2           51.36         102.72  2014-09-23   
6930                      6           85.60         599.20  2014-09-25   
7260                     14           68.48        1968.80  2014-09-19   

                        
            LAST_VISIT  
CUSTOMER_ID             
488         2014-10-03  
5194        2014-10-09  
5398        2014-10-07  
6930        2014-10-15  
7260        2014-10-19

In [27]:
overall_detail.columns

MultiIndex(levels=[['SALES_AMOUNT', 'SALES_DATE'], ['FIRST_VISIT', 'HISTORIC_SALES', 'HISTORIC_VISITS', 'LAST_VISIT', 'MAX_SALESAMOUNT', 'MIN_SALESAMOUNT', 'STD_SALESAMOUNT', 'VARIATION_SALESAMOUNT']],
           labels=[[0, 0, 0, 0, 0, 0, 1, 1], [4, 7, 6, 2, 5, 1, 0, 3]])

In [28]:
overall_detail.columns.droplevel()

Index(['MAX_SALESAMOUNT', 'VARIATION_SALESAMOUNT', 'STD_SALESAMOUNT',
       'HISTORIC_VISITS', 'MIN_SALESAMOUNT', 'HISTORIC_SALES', 'FIRST_VISIT',
       'LAST_VISIT'],
      dtype='object')

In [29]:
overall_detail.columns = overall_detail.columns.droplevel()

In [30]:
overall_detail.head()

,MAX_SALESAMOUNT,VARIATION_SALESAMOUNT,STD_SALESAMOUNT,HISTORIC_VISITS,MIN_SALESAMOUNT,HISTORIC_SALES,FIRST_VISIT,LAST_VISIT
CUSTOMER_ID,,,,,,,,
488,171.20,NaN,NaN,1,171.20,171.20,2014-10-03,2014-10-03
5194,599.20,NaN,NaN,1,599.20,599.20,2014-10-09,2014-10-09
5398,51.36,0.000000,0.000000,2,51.36,102.72,2014-09-23,2014-10-07
6930,171.20,1221.226667,34.946054,6,85.60,599.20,2014-09-25,2014-10-15
7260,256.80,2803.721705,52.950181,14,68.48,1968.80,2014-09-19,2014-10-19


In [31]:
overall_detail.reset_index(level=0, inplace=True)

In [32]:
overall_detail.head()

,CUSTOMER_ID,MAX_SALESAMOUNT,VARIATION_SALESAMOUNT,STD_SALESAMOUNT,HISTORIC_VISITS,MIN_SALESAMOUNT,HISTORIC_SALES,FIRST_VISIT,LAST_VISIT
0,488,171.20,NaN,NaN,1,171.20,171.20,2014-10-03,2014-10-03
1,5194,599.20,NaN,NaN,1,599.20,599.20,2014-10-09,2014-10-09
2,5398,51.36,0.000000,0.000000,2,51.36,102.72,2014-09-23,2014-10-07
3,6930,171.20,1221.226667,34.946054,6,85.60,599.20,2014-09-25,2014-10-15
4,7260,256.80,2803.721705,52.950181,14,68.48,1968.80,2014-09-19,2014-10-19


In [33]:
weeks.reset_index(level=0, inplace=True)

In [34]:
weeks.head()

,CUSTOMER_ID,W0_HISTORIC_VISITS,W1_HISTORIC_VISITS,W2_HISTORIC_VISITS,W3_HISTORIC_VISITS,W4_HISTORIC_VISITS,W5_HISTORIC_VISITS,W0_HISTORIC_SALES,W1_HISTORIC_SALES,W2_HISTORIC_SALES,...,W2_MIN_SALESAMOUNT,W3_MIN_SALESAMOUNT,W4_MIN_SALESAMOUNT,W5_MIN_SALESAMOUNT,W0_MAX_SALESAMOUNT,W1_MAX_SALESAMOUNT,W2_MAX_SALESAMOUNT,W3_MAX_SALESAMOUNT,W4_MAX_SALESAMOUNT,W5_MAX_SALESAMOUNT
0,488,NaN,NaN,NaN,171.2,NaN,NaN,NaN,NaN,NaN,...,NaN,171.2,NaN,NaN,NaN,NaN,NaN,171.2,NaN,NaN
1,5194,NaN,NaN,599.20,NaN,NaN,NaN,NaN,NaN,NaN,...,599.20,NaN,NaN,NaN,NaN,NaN,599.20,NaN,NaN,NaN
2,5398,51.36,NaN,51.36,NaN,51.36,NaN,NaN,NaN,NaN,...,51.36,NaN,51.36,NaN,51.36,NaN,51.36,NaN,51.36,NaN
3,6930,85.60,85.6,171.20,85.6,85.60,NaN,0.00,NaN,3663.680000,...,85.60,85.6,85.60,NaN,256.80,85.60,256.80,85.6,171.20,NaN
4,7260,171.20,171.2,171.20,256.8,171.20,171.2,3663.68,2637.8496,390.792533,...,136.96,85.6,85.60,171.2,256.80,582.08,445.12,513.6,256.80,171.2


In [35]:
overall_detail.head()

,CUSTOMER_ID,MAX_SALESAMOUNT,VARIATION_SALESAMOUNT,STD_SALESAMOUNT,HISTORIC_VISITS,MIN_SALESAMOUNT,HISTORIC_SALES,FIRST_VISIT,LAST_VISIT
0,488,171.20,NaN,NaN,1,171.20,171.20,2014-10-03,2014-10-03
1,5194,599.20,NaN,NaN,1,599.20,599.20,2014-10-09,2014-10-09
2,5398,51.36,0.000000,0.000000,2,51.36,102.72,2014-09-23,2014-10-07
3,6930,171.20,1221.226667,34.946054,6,85.60,599.20,2014-09-25,2014-10-15
4,7260,256.80,2803.721705,52.950181,14,68.48,1968.80,2014-09-19,2014-10-19


Merging Overall_detail and Weeks dataframes.

Note: Total customers should be 200,000

In [36]:
final = overall_detail.merge(weeks, left_on='CUSTOMER_ID', right_on='CUSTOMER_ID', how='outer')

In [37]:
final.shape

(200000, 45)

Checking missing values

In [38]:
final.isnull().sum()

CUSTOMER_ID                       0
MAX_SALESAMOUNT                8158
VARIATION_SALESAMOUNT         53666
STD_SALESAMOUNT               53666
HISTORIC_VISITS                8158
MIN_SALESAMOUNT                8158
HISTORIC_SALES                 8158
FIRST_VISIT                    8158
LAST_VISIT                     8158
W0_HISTORIC_VISITS            89431
W1_HISTORIC_VISITS            88812
W2_HISTORIC_VISITS            87265
W3_HISTORIC_VISITS            79858
W4_HISTORIC_VISITS            89495
W5_HISTORIC_VISITS           108301
W0_HISTORIC_SALES            156644
W1_HISTORIC_SALES            155990
W2_HISTORIC_SALES            155380
W3_HISTORIC_SALES            148376
W4_HISTORIC_SALES            156839
W5_HISTORIC_SALES            173180
W0_STD_SALESAMOUNT           156644
W1_STD_SALESAMOUNT           155990
W2_STD_SALESAMOUNT           155380
W3_STD_SALESAMOUNT           148376
W4_STD_SALESAMOUNT           156839
W5_STD_SALESAMOUNT           173180
W0_VARIATION_SALES_AMOUNT   

Dealing with dates missing values in FIRST_VISIT and LAST_VISIT.

Note: we are observing these missing values becuase 8158 customers didn't come to store from Week-5 to Week-1, they only came in Week-0, so we are replacing missing values with 19-Oct-2014 (The end date of Week-1)

In [39]:
from datetime import datetime
d0 = datetime(2014,10,19)

In [40]:
final.loc[final['FIRST_VISIT'].isnull(),'FIRST_VISIT']=d0

In [41]:
final.loc[final['LAST_VISIT'].isnull(),'LAST_VISIT']=d0

In [42]:
final.isnull().sum()

CUSTOMER_ID                       0
MAX_SALESAMOUNT                8158
VARIATION_SALESAMOUNT         53666
STD_SALESAMOUNT               53666
HISTORIC_VISITS                8158
MIN_SALESAMOUNT                8158
HISTORIC_SALES                 8158
FIRST_VISIT                       0
LAST_VISIT                        0
W0_HISTORIC_VISITS            89431
W1_HISTORIC_VISITS            88812
W2_HISTORIC_VISITS            87265
W3_HISTORIC_VISITS            79858
W4_HISTORIC_VISITS            89495
W5_HISTORIC_VISITS           108301
W0_HISTORIC_SALES            156644
W1_HISTORIC_SALES            155990
W2_HISTORIC_SALES            155380
W3_HISTORIC_SALES            148376
W4_HISTORIC_SALES            156839
W5_HISTORIC_SALES            173180
W0_STD_SALESAMOUNT           156644
W1_STD_SALESAMOUNT           155990
W2_STD_SALESAMOUNT           155380
W3_STD_SALESAMOUNT           148376
W4_STD_SALESAMOUNT           156839
W5_STD_SALESAMOUNT           173180
W0_VARIATION_SALES_AMOUNT   

In remaining columns, we are observing missing values beacause customer did not purchase anything from Store, so technically Sales was ZERO. Hence replacing remaining missing values with ZERO

In [43]:
final.fillna(0, inplace=True)

In [44]:
final.isnull().sum()

CUSTOMER_ID                  0
MAX_SALESAMOUNT              0
VARIATION_SALESAMOUNT        0
STD_SALESAMOUNT              0
HISTORIC_VISITS              0
MIN_SALESAMOUNT              0
HISTORIC_SALES               0
FIRST_VISIT                  0
LAST_VISIT                   0
W0_HISTORIC_VISITS           0
W1_HISTORIC_VISITS           0
W2_HISTORIC_VISITS           0
W3_HISTORIC_VISITS           0
W4_HISTORIC_VISITS           0
W5_HISTORIC_VISITS           0
W0_HISTORIC_SALES            0
W1_HISTORIC_SALES            0
W2_HISTORIC_SALES            0
W3_HISTORIC_SALES            0
W4_HISTORIC_SALES            0
W5_HISTORIC_SALES            0
W0_STD_SALESAMOUNT           0
W1_STD_SALESAMOUNT           0
W2_STD_SALESAMOUNT           0
W3_STD_SALESAMOUNT           0
W4_STD_SALESAMOUNT           0
W5_STD_SALESAMOUNT           0
W0_VARIATION_SALES_AMOUNT    0
W1_VARIATION_SALES_AMOUNT    0
W2_VARIATION_SALES_AMOUNT    0
W3_VARIATION_SALES_AMOUNT    0
W4_VARIATION_SALES_AMOUNT    0
W5_VARIA

Creating CHURN column from Week-0

In [45]:
final[['W0_HISTORIC_VISITS', 'W0_HISTORIC_SALES', 'W0_STD_SALESAMOUNT', 'W0_VARIATION_SALES_AMOUNT', 'W0_MIN_SALESAMOUNT', 'W0_MAX_SALESAMOUNT']].head()

,W0_HISTORIC_VISITS,W0_HISTORIC_SALES,W0_STD_SALESAMOUNT,W0_VARIATION_SALES_AMOUNT,W0_MIN_SALESAMOUNT,W0_MAX_SALESAMOUNT
0,0.00,0.00,0.00000,0.0,0.00,0.00
1,0.00,0.00,0.00000,0.0,0.00,0.00
2,51.36,0.00,0.00000,1.0,51.36,51.36
3,85.60,0.00,0.00000,3.0,85.60,256.80
4,171.20,3663.68,60.52834,2.0,85.60,256.80


In [46]:
def identifyChurn(sale):
    if sale > 0:
        return 0
    else:
        return 1

In [47]:
final['CHURN'] = 0

In [48]:
final.loc[:, ('CHURN')] = final['W0_HISTORIC_SALES'].apply(identifyChurn)

In [49]:
final.columns

Index(['CUSTOMER_ID', 'MAX_SALESAMOUNT', 'VARIATION_SALESAMOUNT',
       'STD_SALESAMOUNT', 'HISTORIC_VISITS', 'MIN_SALESAMOUNT',
       'HISTORIC_SALES', 'FIRST_VISIT', 'LAST_VISIT', 'W0_HISTORIC_VISITS',
       'W1_HISTORIC_VISITS', 'W2_HISTORIC_VISITS', 'W3_HISTORIC_VISITS',
       'W4_HISTORIC_VISITS', 'W5_HISTORIC_VISITS', 'W0_HISTORIC_SALES',
       'W1_HISTORIC_SALES', 'W2_HISTORIC_SALES', 'W3_HISTORIC_SALES',
       'W4_HISTORIC_SALES', 'W5_HISTORIC_SALES', 'W0_STD_SALESAMOUNT',
       'W1_STD_SALESAMOUNT', 'W2_STD_SALESAMOUNT', 'W3_STD_SALESAMOUNT',
       'W4_STD_SALESAMOUNT', 'W5_STD_SALESAMOUNT', 'W0_VARIATION_SALES_AMOUNT',
       'W1_VARIATION_SALES_AMOUNT', 'W2_VARIATION_SALES_AMOUNT',
       'W3_VARIATION_SALES_AMOUNT', 'W4_VARIATION_SALES_AMOUNT',
       'W5_VARIATION_SALES_AMOUNT', 'W0_MIN_SALESAMOUNT', 'W1_MIN_SALESAMOUNT',
       'W2_MIN_SALESAMOUNT', 'W3_MIN_SALESAMOUNT', 'W4_MIN_SALESAMOUNT',
       'W5_MIN_SALESAMOUNT', 'W0_MAX_SALESAMOUNT', 'W1_MAX_SALESAMOUNT',

Dropping Week-0 remaining features after creating CHURN column

In [50]:
final.drop(['W0_HISTORIC_VISITS', 'W0_HISTORIC_SALES', 'W0_MAX_SALESAMOUNT', 'W0_VARIATION_SALES_AMOUNT', 'W0_STD_SALESAMOUNT', 'W0_MIN_SALESAMOUNT'], axis=1, inplace=True)

Converting FIRST_VIST and LAST_VIST into number days

In [51]:
final['FIRST_VISIT_DAY'] = 4444
final.loc[:, ('FIRST_VISIT_DAY')] = d0 - final['FIRST_VISIT']
final['LAST_VISIT_DAY'] = 4444
final.loc[:, ('LAST_VISIT_DAY')] = d0 - final['LAST_VISIT']

In [52]:
final.head()

,CUSTOMER_ID,MAX_SALESAMOUNT,VARIATION_SALESAMOUNT,STD_SALESAMOUNT,HISTORIC_VISITS,MIN_SALESAMOUNT,HISTORIC_SALES,FIRST_VISIT,LAST_VISIT,W1_HISTORIC_VISITS,...,W4_MIN_SALESAMOUNT,W5_MIN_SALESAMOUNT,W1_MAX_SALESAMOUNT,W2_MAX_SALESAMOUNT,W3_MAX_SALESAMOUNT,W4_MAX_SALESAMOUNT,W5_MAX_SALESAMOUNT,CHURN,FIRST_VISIT_DAY,LAST_VISIT_DAY
0,488,171.20,0.000000,0.000000,1.0,171.20,171.20,2014-10-03,2014-10-03,0.0,...,0.00,0.0,0.00,0.00,171.2,0.00,0.0,1,16 days,16 days
1,5194,599.20,0.000000,0.000000,1.0,599.20,599.20,2014-10-09,2014-10-09,0.0,...,0.00,0.0,0.00,599.20,0.0,0.00,0.0,1,10 days,10 days
2,5398,51.36,0.000000,0.000000,2.0,51.36,102.72,2014-09-23,2014-10-07,0.0,...,51.36,0.0,0.00,51.36,0.0,51.36,0.0,1,26 days,12 days
3,6930,171.20,1221.226667,34.946054,6.0,85.60,599.20,2014-09-25,2014-10-15,85.6,...,85.60,0.0,85.60,256.80,85.6,171.20,0.0,1,24 days,4 days
4,7260,256.80,2803.721705,52.950181,14.0,68.48,1968.80,2014-09-19,2014-10-19,171.2,...,85.60,171.2,582.08,445.12,513.6,256.80,171.2,0,30 days,0 days


In [53]:
final.dtypes

CUSTOMER_ID                            int64
MAX_SALESAMOUNT                      float64
VARIATION_SALESAMOUNT                float64
STD_SALESAMOUNT                      float64
HISTORIC_VISITS                      float64
MIN_SALESAMOUNT                      float64
HISTORIC_SALES                       float64
FIRST_VISIT                   datetime64[ns]
LAST_VISIT                    datetime64[ns]
W1_HISTORIC_VISITS                   float64
W2_HISTORIC_VISITS                   float64
W3_HISTORIC_VISITS                   float64
W4_HISTORIC_VISITS                   float64
W5_HISTORIC_VISITS                   float64
W1_HISTORIC_SALES                    float64
W2_HISTORIC_SALES                    float64
W3_HISTORIC_SALES                    float64
W4_HISTORIC_SALES                    float64
W5_HISTORIC_SALES                    float64
W1_STD_SALESAMOUNT                   float64
W2_STD_SALESAMOUNT                   float64
W3_STD_SALESAMOUNT                   float64
W4_STD_SAL

In [54]:
final['FIRST_VISIT_DAY'] = final['FIRST_VISIT_DAY'].dt.days
final['LAST_VISIT_DAY'] = final['LAST_VISIT_DAY'].dt.days

In [55]:
final.head()

,CUSTOMER_ID,MAX_SALESAMOUNT,VARIATION_SALESAMOUNT,STD_SALESAMOUNT,HISTORIC_VISITS,MIN_SALESAMOUNT,HISTORIC_SALES,FIRST_VISIT,LAST_VISIT,W1_HISTORIC_VISITS,...,W4_MIN_SALESAMOUNT,W5_MIN_SALESAMOUNT,W1_MAX_SALESAMOUNT,W2_MAX_SALESAMOUNT,W3_MAX_SALESAMOUNT,W4_MAX_SALESAMOUNT,W5_MAX_SALESAMOUNT,CHURN,FIRST_VISIT_DAY,LAST_VISIT_DAY
0,488,171.20,0.000000,0.000000,1.0,171.20,171.20,2014-10-03,2014-10-03,0.0,...,0.00,0.0,0.00,0.00,171.2,0.00,0.0,1,16,16
1,5194,599.20,0.000000,0.000000,1.0,599.20,599.20,2014-10-09,2014-10-09,0.0,...,0.00,0.0,0.00,599.20,0.0,0.00,0.0,1,10,10
2,5398,51.36,0.000000,0.000000,2.0,51.36,102.72,2014-09-23,2014-10-07,0.0,...,51.36,0.0,0.00,51.36,0.0,51.36,0.0,1,26,12
3,6930,171.20,1221.226667,34.946054,6.0,85.60,599.20,2014-09-25,2014-10-15,85.6,...,85.60,0.0,85.60,256.80,85.6,171.20,0.0,1,24,4
4,7260,256.80,2803.721705,52.950181,14.0,68.48,1968.80,2014-09-19,2014-10-19,171.2,...,85.60,171.2,582.08,445.12,513.6,256.80,171.2,0,30,0


Dropping FINAL_VISIT and LAST_VISIT after creating number of days features

In [56]:
final.drop(['LAST_VISIT','FIRST_VISIT'],axis=1,inplace=True)

In [57]:
#final.to_csv('ADS_Final_Unscaled.csv')

Checking variance of each column

In [58]:
final.var()

CUSTOMER_ID                  2.339707e+17
MAX_SALESAMOUNT              7.611315e+04
VARIATION_SALESAMOUNT        4.484922e+11
STD_SALESAMOUNT              7.375263e+03
HISTORIC_VISITS              1.698200e+01
MIN_SALESAMOUNT              4.604913e+04
HISTORIC_SALES               5.019255e+05
W1_HISTORIC_VISITS           2.705151e+04
W2_HISTORIC_VISITS           2.752446e+04
W3_HISTORIC_VISITS           3.235890e+04
W4_HISTORIC_VISITS           2.175563e+04
W5_HISTORIC_VISITS           2.420888e+04
W1_HISTORIC_SALES            8.199242e+08
W2_HISTORIC_SALES            2.231625e+10
W3_HISTORIC_SALES            4.230645e+08
W4_HISTORIC_SALES            7.160878e+08
W5_HISTORIC_SALES            3.087314e+08
W1_STD_SALESAMOUNT           1.289380e+03
W2_STD_SALESAMOUNT           1.697914e+03
W3_STD_SALESAMOUNT           1.537366e+03
W4_STD_SALESAMOUNT           1.137992e+03
W5_STD_SALESAMOUNT           6.967622e+02
W1_VARIATION_SALES_AMOUNT    1.187531e+00
W2_VARIATION_SALES_AMOUNT    1.194

Performing log transformation on columns with variance higher than 500

In [59]:
feat_var = final.var()

In [60]:
tmp = pd.DataFrame({"feature":feat_var.index,"var":feat_var.values})

In [61]:
tmp=tmp.sort_values('var',ascending=False)
tmp

,feature,var
0,CUSTOMER_ID,2.339707e+17
2,VARIATION_SALESAMOUNT,4.484922e+11
13,W2_HISTORIC_SALES,2.231625e+10
12,W1_HISTORIC_SALES,8.199242e+08
15,W4_HISTORIC_SALES,7.160878e+08
14,W3_HISTORIC_SALES,4.230645e+08
16,W5_HISTORIC_SALES,3.087314e+08
6,HISTORIC_SALES,5.019255e+05
1,MAX_SALESAMOUNT,7.611315e+04
34,W3_MAX_SALESAMOUNT,5.493128e+04


dropping CUSTOMER_ID and CHURN

In [62]:
tmp.drop([0,37],axis=0,inplace=True)

In [63]:
feat_log=tmp[tmp['var']>500]['feature']
feat_log.reset_index(drop=True)

0     VARIATION_SALESAMOUNT
1         W2_HISTORIC_SALES
2         W1_HISTORIC_SALES
3         W4_HISTORIC_SALES
4         W3_HISTORIC_SALES
5         W5_HISTORIC_SALES
6            HISTORIC_SALES
7           MAX_SALESAMOUNT
8        W3_MAX_SALESAMOUNT
9        W2_MAX_SALESAMOUNT
10          MIN_SALESAMOUNT
11       W1_MAX_SALESAMOUNT
12       W4_MAX_SALESAMOUNT
13       W3_HISTORIC_VISITS
14       W5_MAX_SALESAMOUNT
15       W2_HISTORIC_VISITS
16       W3_MIN_SALESAMOUNT
17       W1_HISTORIC_VISITS
18       W5_HISTORIC_VISITS
19       W1_MIN_SALESAMOUNT
20       W2_MIN_SALESAMOUNT
21       W5_MIN_SALESAMOUNT
22       W4_HISTORIC_VISITS
23       W4_MIN_SALESAMOUNT
24          STD_SALESAMOUNT
25       W2_STD_SALESAMOUNT
26       W3_STD_SALESAMOUNT
27       W1_STD_SALESAMOUNT
28       W4_STD_SALESAMOUNT
29       W5_STD_SALESAMOUNT
Name: feature, dtype: object

In [64]:
df_logtransf=final.copy()

In [65]:
for i in feat_log:
    df_logtransf[i]=np.log(1+df_logtransf[i])

In [66]:
df_logtransf.var()

CUSTOMER_ID                  2.339707e+17
MAX_SALESAMOUNT              1.446312e+00
VARIATION_SALESAMOUNT        1.531214e+01
STD_SALESAMOUNT              3.863284e+00
HISTORIC_VISITS              1.698200e+01
MIN_SALESAMOUNT              1.160588e+00
HISTORIC_SALES               2.420955e+00
W1_HISTORIC_VISITS           5.978541e+00
W2_HISTORIC_VISITS           5.969099e+00
W3_HISTORIC_VISITS           5.907494e+00
W4_HISTORIC_VISITS           5.954661e+00
W5_HISTORIC_VISITS           5.854975e+00
W1_HISTORIC_SALES            6.938760e+00
W2_HISTORIC_SALES            7.006650e+00
W3_HISTORIC_SALES            8.289329e+00
W4_HISTORIC_SALES            6.708418e+00
W5_HISTORIC_SALES            4.238075e+00
W1_STD_SALESAMOUNT           1.758614e+00
W2_STD_SALESAMOUNT           1.775726e+00
W3_STD_SALESAMOUNT           2.098810e+00
W4_STD_SALESAMOUNT           1.700909e+00
W5_STD_SALESAMOUNT           1.075015e+00
W1_VARIATION_SALES_AMOUNT    1.187531e+00
W2_VARIATION_SALES_AMOUNT    1.194

In [67]:
df_logtransf.head()

,CUSTOMER_ID,MAX_SALESAMOUNT,VARIATION_SALESAMOUNT,STD_SALESAMOUNT,HISTORIC_VISITS,MIN_SALESAMOUNT,HISTORIC_SALES,W1_HISTORIC_VISITS,W2_HISTORIC_VISITS,W3_HISTORIC_VISITS,...,W4_MIN_SALESAMOUNT,W5_MIN_SALESAMOUNT,W1_MAX_SALESAMOUNT,W2_MAX_SALESAMOUNT,W3_MAX_SALESAMOUNT,W4_MAX_SALESAMOUNT,W5_MAX_SALESAMOUNT,CHURN,FIRST_VISIT_DAY,LAST_VISIT_DAY
0,488,5.148657,0.00000,0.000000,1.0,5.148657,5.148657,0.000000,0.000000,5.148657,...,0.000000,0.000000,0.000000,0.000000,5.148657,0.000000,0.000000,1,16,16
1,5194,6.397263,0.00000,0.000000,1.0,6.397263,6.397263,0.000000,6.397263,0.000000,...,0.000000,0.000000,0.000000,6.397263,0.000000,0.000000,0.000000,1,10,10
2,5398,3.958143,0.00000,0.000000,2.0,3.958143,4.641695,0.000000,3.958143,0.000000,...,3.958143,0.000000,0.000000,3.958143,0.000000,3.958143,0.000000,1,26,12
3,6930,5.148657,7.10843,3.582019,6.0,4.461300,6.397263,4.461300,5.148657,4.461300,...,4.461300,0.000000,4.461300,5.552184,4.461300,5.148657,0.000000,1,24,4
4,7260,5.552184,7.93906,3.988061,14.0,4.241039,7.585687,5.148657,5.148657,5.552184,...,4.461300,5.148657,6.368324,6.100588,6.243390,5.552184,5.148657,0,30,0


Final ADS ready

In [68]:
final_ADS = df_logtransf.copy()

In [69]:
final_ADS.head()

,CUSTOMER_ID,MAX_SALESAMOUNT,VARIATION_SALESAMOUNT,STD_SALESAMOUNT,HISTORIC_VISITS,MIN_SALESAMOUNT,HISTORIC_SALES,W1_HISTORIC_VISITS,W2_HISTORIC_VISITS,W3_HISTORIC_VISITS,...,W4_MIN_SALESAMOUNT,W5_MIN_SALESAMOUNT,W1_MAX_SALESAMOUNT,W2_MAX_SALESAMOUNT,W3_MAX_SALESAMOUNT,W4_MAX_SALESAMOUNT,W5_MAX_SALESAMOUNT,CHURN,FIRST_VISIT_DAY,LAST_VISIT_DAY
0,488,5.148657,0.00000,0.000000,1.0,5.148657,5.148657,0.000000,0.000000,5.148657,...,0.000000,0.000000,0.000000,0.000000,5.148657,0.000000,0.000000,1,16,16
1,5194,6.397263,0.00000,0.000000,1.0,6.397263,6.397263,0.000000,6.397263,0.000000,...,0.000000,0.000000,0.000000,6.397263,0.000000,0.000000,0.000000,1,10,10
2,5398,3.958143,0.00000,0.000000,2.0,3.958143,4.641695,0.000000,3.958143,0.000000,...,3.958143,0.000000,0.000000,3.958143,0.000000,3.958143,0.000000,1,26,12
3,6930,5.148657,7.10843,3.582019,6.0,4.461300,6.397263,4.461300,5.148657,4.461300,...,4.461300,0.000000,4.461300,5.552184,4.461300,5.148657,0.000000,1,24,4
4,7260,5.552184,7.93906,3.988061,14.0,4.241039,7.585687,5.148657,5.148657,5.552184,...,4.461300,5.148657,6.368324,6.100588,6.243390,5.552184,5.148657,0,30,0


In [70]:
#final_ADS.to_csv('ADS_Chappa_Out')